In [1]:
import numpy as np
import gym
import random
import math

In [16]:
env1 =["SHFF", "FFFH", "FHFH", "HFFG"]
env2= ["SFFFFF", "FFFHFF", "FHFHHH", "HFFFFG"]
env3 = ['SFFHFFHH', 'HFFFFFHF', 'HFFHHFHH', 'HFHHHFFF', 'HFHHFHFF', 'FFFFFFFH', 'FHHFHFHH', 'FHHFHFFG'] 

selectedEnv = env2
env = gym.make('FrozenLake-v1', desc=selectedEnv, render_mode="rgb_array", is_slippery = False)
env.reset()
env.render()

array([[[180, 200, 230],
        [180, 200, 230],
        [180, 200, 230],
        ...,
        [180, 200, 230],
        [180, 200, 230],
        [180, 200, 230]],

       [[180, 200, 230],
        [204, 230, 255],
        [204, 230, 255],
        ...,
        [204, 230, 255],
        [204, 230, 255],
        [180, 200, 230]],

       [[180, 200, 230],
        [235, 245, 249],
        [204, 230, 255],
        ...,
        [204, 230, 255],
        [204, 230, 255],
        [180, 200, 230]],

       ...,

       [[180, 200, 230],
        [235, 245, 249],
        [235, 245, 249],
        ...,
        [204, 230, 255],
        [235, 245, 249],
        [180, 200, 230]],

       [[180, 200, 230],
        [235, 245, 249],
        [235, 245, 249],
        ...,
        [204, 230, 255],
        [204, 230, 255],
        [180, 200, 230]],

       [[180, 200, 230],
        [180, 200, 230],
        [180, 200, 230],
        ...,
        [180, 200, 230],
        [180, 200, 230],
        [180, 200, 230]]

In [3]:
# change-able parameters:
discount_factor = 0.99
delta_threshold = 0.00001
epsilon = 1

In [14]:
def value_iteration(env, gamma=0.9, epsilon=1e-6):
    num_states = env.observation_space.n
    num_actions = env.action_space.n

    # Initialize the value function for all states as 0
    V = np.zeros(num_states)
    
    # Learning the policy here
    while True:
        delta = 0
        for state in range(num_states):
            v = V[state] #value of the current state
            # Applying the Bellman equation to update the value of the state
            temp_values = np.zeros(num_actions) 
            for action in range(num_actions): #for each action
                for prob, next_state, reward, _ in env.P[state][action]: #for each possible transition
                    temp_values[action] += prob * (reward + gamma * V[next_state]) #summation of the expected value of the next state
             #Updating the value of the state by choosing action that maximizes the expected value of the next state
            V[state] = np.max(temp_values)
            delta = max(delta, np.abs(v - V[state])) #calculating the change in value of the state
        if delta < epsilon: #if the change in value of the state is less than the threshold the optimal plocy is learnt
            break

    #Code for extracting optimal policy from the value function

    policy = np.zeros(num_states)   #initializing the policy
    for state in range(num_states): #for each state
        temp_values = np.zeros(num_actions) 
        for action in range(num_actions):  #for each action
            for prob, next_state, reward,_ in env.P[state][action]: #for each possible transition
                temp_values[action] += prob * (reward + gamma * V[next_state]) #summation of the expected value of the next state
        policy[state] = np.argmax(temp_values) #selecting the action that maximizes the expected value of the next state
    
    return policy, V

In [15]:
# Run value iteration
policy, V = value_iteration(env, gamma=discount_factor, epsilon=delta_threshold)

# Print results
print("Optimal Value Function:")
print(V.reshape(len(selectedEnv), len(selectedEnv[0])))

print("\nOptimal Policy (0=Left, 1=Down, 2=Right, 3=Up):")
print(policy.reshape(len(selectedEnv), len(selectedEnv[0])))

Optimal Value Function:
[[0.93206535 0.94148015 0.95099005 0.94148015 0.93206535 0.92274469]
 [0.94148015 0.95099005 0.96059601 0.         0.92274469 0.91351725]
 [0.93206535 0.         0.970299   0.         0.         0.        ]
 [0.         0.970299   0.9801     0.99       1.         0.        ]]

Optimal Policy (0=Left, 1=Down, 2=Right, 3=Up):
[[1. 1. 1. 0. 0. 0.]
 [2. 2. 1. 0. 3. 0.]
 [3. 0. 1. 0. 0. 0.]
 [0. 2. 2. 2. 2. 0.]]


In [6]:
# resetting the environment and executing the policy
state = env.reset()
state = state[0]
step = 0
done = False
print(state)

max_steps = 100
for step in range(max_steps):

    # Getting max value against that state, so that we choose that action
    action = policy[state]
    new_state, reward, done, truncated, info = env.step(action) #information after taking the action
    env.render()
    if done:
        print("number of steps taken:", step)
        break

    state = new_state

env.close()


0
number of steps taken: 7


c:\Users\HP\miniconda3\envs\env\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
